<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/archeology_25june2025A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [ ]:
# Based on similar location - modified one to include weights collecting different geographic layers
# to provide to the deep learning model for training
import geemap

# Define region (Brazil, Southeastern Amazon). You can also expand it to other countries
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na', 'Brazil'))
brazil_geometry = mask.geometry()
southeast_brazil = brazil_geometry.intersection(ee.Geometry.Rectangle([-55, -25, -40, -35]))
# weights can be adjusted during the model training. The task can be automated using an optimization algorithms
weights = {
    "B12": 0.10,
    "B11": 0.10,
    "B8": 0.10,
    "VV": 0.10,
    "NDVI": 0.15,
    "Elevation": 0.15,
    "SoilMoisture": 0.10,
    "Thermal": 0.1,
    "Evapotranspiration": 0.1
}
def normalize_band(image, band):
    band_min = image.select(band).reduceRegion(
        reducer=ee.Reducer.min(), geometry=image.geometry(), scale=30, bestEffort=True
    ).get(band)

    band_max = image.select(band).reduceRegion(
        reducer=ee.Reducer.max(), geometry=image.geometry(), scale=30, bestEffort=True
    ).get(band)

    normalized_band = image.select(band).subtract(ee.Number(band_min)).divide(ee.Number(band_max).subtract(ee.Number(band_min)))
    return normalized_band.rename(band + "_norm")
# Load archaeological points
archaeology_points = ee.FeatureCollection("users/mohamadawadlebanon/Archeologicalsites")

# Define date range
start_date = "2024-05-01"
end_date = "2024-05-31"
def apply_scale_and_offset(image):
    return image.select("ST_B10").multiply(0.00341802).add(149.0)
### **Step 1: Elevation Data (ASTER GDEM)**
elevation = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").clip(southeast_brazil)
#elevation = elevation.rename("Elevation")
elevation = elevation.rename("elevation")
### **Step 2: Sentinel-2 EVI**
sentinel2_ndvi = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .select(["B8", "B4"]) \
    .map(lambda img: img.expression(
        "(B8 - B4) / (B8 + B4)",  # EVI formula
        {"B8": img.select("B8"), "B4": img.select("B4")}
    )).median().clip(southeast_brazil)
sentinel2_ndvi = sentinel2_ndvi.rename("NDVI")

### **Step 3: Sentinel-1 Radar**
sentinel1_images = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
    .filter(ee.Filter.eq("instrumentMode", "IW")) \
    .select("VV")

mosaiced_sentinel1 = sentinel1_images.median().clip(southeast_brazil)

### **Step 4: Sentinel-2 Optical Mosaic**
sentinel2_images = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10)) \
    .select(["B8", "B11", "B12"])

mosaiced_sentinel2 = sentinel2_images.median().clip(southeast_brazil)

### **Step 5: MODIS Evapotranspiration (ET)**
modis_et = ee.ImageCollection("MODIS/061/MOD16A2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .select("ET") \
    .median().clip(southeast_brazil)

### **Step 6: Soil Moisture (NASA SMAP)**
soil_moisture = ee.ImageCollection('NASA/SMAP/SPL4SMGP/007') \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .select('sm_surface') \
    .median().clip(southeast_brazil)

### **Step 7: Thermal Infrared (Landsat 8-9)**
thermal = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
    .filterBounds(southeast_brazil) \
    .filterDate(start_date, end_date) \
    .map(apply_scale_and_offset) \
    .median().clip(southeast_brazil)


#*** step 8 Normalize all bands

normalized_sentinel1 = normalize_band(mosaiced_sentinel1, "VV")
normalized_ndvi = normalize_band(sentinel2_ndvi, "NDVI")
normalized_elevation = normalize_band(elevation, "elevation")
normalized_soil_moisture = normalize_band(soil_moisture, "sm_surface")
normalized_thermal = normalize_band(thermal, "ST_B10")
normalized_et = normalize_band(modis_et, "ET")

### merge all bands and layers
combined = mosaiced_sentinel2.select("B12").multiply(weights["B12"]) \
.addBands(mosaiced_sentinel2.select("B11").multiply(weights["B11"])) \
.addBands(mosaiced_sentinel2.select("B8").multiply(weights["B8"])) \
.addBands(mosaiced_sentinel1.multiply(weights["VV"])) \
.addBands(sentinel2_ndvi.multiply(weights["NDVI"])) \
.addBands(elevation.multiply(weights["Elevation"])) \
.addBands(soil_moisture.multiply(weights["SoilMoisture"])) \
.addBands(thermal.multiply(weights["Thermal"])) \
.addBands(modis_et.multiply(weights["Evapotranspiration"]))


# Sample feature values at archaeology site locations
sampled_values = combined.sampleRegions(**{
    "collection": archaeology_points,
    "scale": 10,  # Adjust scale depending on resolution needs
    "tileScale": 2
})
# Convert sampled values to FeatureCollection table
def safe_set_coordinates(feature):
    return feature.set({
        "Longitude": feature.get("lon"),
        "Latitude": feature.get("lat")
    })

#table = sampled_values.map(safe_set_coordinates)

task = ee.batch.Export.table.toDrive(
    collection=sampled_values, #table,
    description="Archaeological_Site_Features",
    fileFormat="CSV"
)
task.start()  # Start export task

### **Step 10: Apply K-Means Clustering**
num_classes = 10
training_points = combined.sample(
    region=southeast_brazil,
    # Default (False) is no geometries in the output.
    # When set to True, each feature has a Point geometry at the center of the
    # image pixel.
    geometries=True,
    numPixels = 500,
    # The scale is not specified, so the resolution of the image will be used,
    # and there is a feature for every pixel. If we give a scale parameter, the
    # image will be resampled and there will be more or fewer features.
    #
    scale= 10,
)
# Extract longitude and latitude from geometry

#training_points = training_points.filter(ee.Filter.notNull(['geometry']))
print(training_points.first().getInfo())
task1 = ee.batch.Export.table.toDrive(
    collection=training_points,
    description="Samples_random_collection",
    fileFormat="CSV"
)
task1.start()
clusterer = ee.Clusterer.wekaKMeans(num_classes).train(training_points)
classified = combined.cluster(clusterer)

### **Step 11: Validate Using Archaeological Sites**
validation = classified.sampleRegions(**{
    "collection": archaeology_points,
    "scale": 10,
    "properties": ["class"],
    "tileScale": 2
})

### **Step 12: Visualization Parameters**
vis_params_elevation = {"bands": ["elevation"], "min": 0, "max": 3000, "palette": ["black", "white"]}
vis_params_ndvi = {"bands": ["NDVI"],"min": -0.6, "max": 0.6, "palette": ["brown", "green"]}
vis_params_s1 = {"bands": ["VV"], "min": -20, "max": 0, "gamma": 1.4}
vis_params_s2 = {"bands": ["B12", "B11", "B8"], "min": 0, "max": 3000, "gamma": 1.4}
vis_params_modis_et = {"bands": ["ET"], "min": 0, "max": 300, "palette": ["yellow", "green", "blue"]}
#vis_params_pml_et = {"min": 0, "max": 5, "palette": ["orange", "red", "purple"]}
vis_params_soil = {"bands": ["sm_surface"], "min": 0, "max": 0.9, "palette": ["red", "orange", "yellow", "green"]}
vis_params_thermal = {"bands": ["ST_B10"], "min": 270, "max": 320, "palette": ["blue", "yellow", "red"]}
vis_params_classified = {
    "min": 0,
    "max": num_classes - 1,
    "palette": ["blue", "green", "yellow", "red", "purple", "orange", "brown", "cyan", "pink", "gray"]
}
archaeology_vis = {"color": "blue", "pointRadius": 5}
sample_vis = {"color": "black", "pointRadius": 4}
### **Step 13: Create & Display Map**
m = geemap.Map(center=[-3, -60], zoom=6)

m.addLayer(elevation, vis_params_elevation, "ASTER GDEM v3 Elevation")
m.addLayer(sentinel2_ndvi, vis_params_ndvi, "High-Resolution NDVI (Sentinel-2)")
m.addLayer(mosaiced_sentinel1, vis_params_s1, "Sentinel-1 Mosaic (Radar)")
m.addLayer(mosaiced_sentinel2, vis_params_s2, "Sentinel-2 Mosaic (Optical)")
m.addLayer(modis_et, vis_params_modis_et, "MODIS Evapotranspiration")
#m.addLayer(pml_et, vis_params_pml_et, "PML Evapotranspiration")
m.addLayer(soil_moisture, vis_params_soil, "Soil Moisture (SMAP)")
m.addLayer(thermal, vis_params_thermal, "Thermal Infrared (Landsat)")
m.addLayer(classified, vis_params_classified, "Classified Image (K-Means)")
m.addLayer(archaeology_points, archaeology_vis, "Archaeology Points")
m.addLayer(training_points,sample_vis, "sample Points")

# Display the map
m


In [ ]:
!pip install geedim

In [ ]:

import geemap
import os

# Set area and output parameters
region = southeast_brazil
scale = 100  # You can fine-tune for better resolution
output_dir = 'exported_maps'
os.makedirs(output_dir, exist_ok=True)

# Ordered thematic layers: (filename prefix, ee.Image or ee.FeatureCollection, visParams)
layers_to_export = [
    ("1_World_Map", ee.Image().paint(ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017"), 0, 2), {"palette": ["white"]}),
    ("2_Study_Area", ee.Image().paint(southeast_brazil, 0, 3), {"palette": ["red"]}),
    ("3_Archaeology_Sites", ee.Image().paint(archaeology_points, 0, 4), {"palette": ["blue"]}),
    ("4_Sentinel2_RGB", mosaiced_sentinel2.visualize(**vis_params_s2), {}),
    ("5_Sentinel1_Radar", mosaiced_sentinel1.visualize(**vis_params_s1), {}),
    ("6_NDVI", sentinel2_ndvi.visualize(**vis_params_ndvi), {}),
    ("7_Elevation", elevation.visualize(**vis_params_elevation), {}),
    ("8_Thermal", thermal.visualize(**vis_params_thermal), {}),
    ("9_Soil_Moisture", soil_moisture.visualize(**vis_params_soil), {}),
    ("10_Segmented", classified.visualize(**vis_params_classified), {}),
    ("11_Predicted_Points", ee.Image().paint(training_points, 0, 5), {"palette": ["black"]})
]
crs = 'EPSG:4326'  # WGS 84, standard geographic projection

for name, image, vis in layers_to_export:
    path = os.path.join(output_dir, f"{name}.png")

    # Handle visualization if not already rendered
    if vis:
        image = image.visualize(**vis)

    # Export with explicit CRS and scale
    geemap.download_ee_image(
        image=image,
        region=region,
        filename=path,
        scale=scale,
        crs=crs
    )
    print(f"Exported {path}")



In [ ]:

import geemap
import os
import numpy as np
import rasterio
from PIL import Image

# Parameters
output_dir = 'exported_maps'
region = southeast_brazil
scale = 100
crs = 'EPSG:4326'
os.makedirs(output_dir, exist_ok=True)

# Utility function: Convert GeoTIFF to displayable PNG
def convert_to_png(input_tif, output_png):
    with rasterio.open(input_tif) as src:
        arr = src.read(1)
        arr = np.where(arr == src.nodata, np.nan, arr)
        # Normalize to 0–255
        norm = (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr))
        norm = np.nan_to_num(norm)
        img = (norm * 255).astype(np.uint8)
        Image.fromarray(img).save(output_png)

# Layers to export
layers_to_export = [
    ("1_World_Map", ee.Image().paint(ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017"), 255, 2), {"palette": ["#d3d3d3"]}),
    ("2_Study_Area", ee.Image().paint(southeast_brazil, 255, 3), {"palette": ["#ff0000"]}),
    ("3_Archaeology_Sites", ee.Image().paint(archaeology_points, 255, 4), {"palette": ["#0000ff"]}),
    ("4_Sentinel2_RGB", mosaiced_sentinel2.visualize(**vis_params_s2), {}),
    ("5_Sentinel1_Radar", mosaiced_sentinel1.visualize(**vis_params_s1), {}),
    ("6_NDVI", sentinel2_ndvi.visualize(**vis_params_ndvi), {}),
    ("7_Elevation", elevation.visualize(**vis_params_elevation), {}),
    ("8_Thermal", thermal.visualize(**vis_params_thermal), {}),
    ("9_Soil_Moisture", soil_moisture.visualize(**vis_params_soil), {}),
    ("10_Segmented", classified.visualize(**vis_params_classified), {}),
    ("11_Predicted_Points", ee.Image().paint(training_points, 255, 5), {"palette": ["#000000"]})
]

for name, image, vis in layers_to_export:
    tif_path = os.path.join(output_dir, f"{name}.tif")
    png_path = os.path.join(output_dir, f"{name}.png")

    # Apply visualization if needed
    if vis:
        image = image.visualize(**vis)

    # Export GeoTIFF
    geemap.download_ee_image(
        image=image,
        region=region,
        filename=tif_path,
        scale=scale,
        crs=crs
    )

    # Convert to true PNG for compatibility
    convert_to_png(tif_path, png_path)
    print(f"✅ Saved: {png_path}")


In [ ]:

import geemap
import os
import numpy as np
import rasterio
from PIL import Image



# Setup
output_dir = 'exported_maps'
region = southeast_brazil
scale = 100
crs = 'EPSG:4326'
os.makedirs(output_dir, exist_ok=True)

# Utility: Convert raster to PNG with optional colormap
def stretch_and_convert(tif_path, png_path, cmap='viridis'):
    with rasterio.open(tif_path) as src:
        arr = src.read(1)
        arr = np.where(arr == src.nodata, np.nan, arr)
        norm = (arr - np.nanmin(arr)) / (np.nanmax(arr) - np.nanmin(arr))
        norm = np.nan_to_num(norm)
        color_map = plt.get_cmap(cmap)
        rgb = (color_map(norm)[:, :, :3] * 255).astype(np.uint8)
        Image.fromarray(rgb).save(png_path)

# Define raw layers
raw_layers = {
    "World_Borders": ee.Image().paint(ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017"), 1, 2),
    "Study_Area": ee.Image().paint(southeast_brazil, 1, 3),
    "Archaeology_Sites": ee.Image().paint(archaeology_points, 1, 4),
    "Sentinel2_Bands": mosaiced_sentinel2.select("B12"),
    "Sentinel1_VV": mosaiced_sentinel1.select("VV"),
    "NDVI": sentinel2_ndvi,
    "Elevation": elevation.select("elevation"),
    "Thermal": thermal.select("ST_B10"),
    "Soil_Moisture": soil_moisture.select("sm_surface"),
    "Segmented": classified,
    "Predicted_Points": ee.Image().paint(training_points, 1, 5)
}

# Optional: assign preferred matplotlib colormaps
colormaps = {
    "World_Borders": "Greys",
    "Study_Area": "Reds",
    "Archaeology_Sites": "Blues",
    "Sentinel2_Bands": "magma",
    "Sentinel1_VV": "Greys",
    "NDVI": "Greens",
    "Elevation": "viridis",
    "Thermal": "coolwarm",
    "Soil_Moisture": "YlOrRd",
    "Segmented": "tab10",
    "Predicted_Points": "Greys"
}

# Export and convert each layer
for name, layer in raw_layers.items():
    tif_path = os.path.join(output_dir, f"{name}.tif")
    png_path = os.path.join(output_dir, f"{name}.png")

    # Ensure it's visualized: raw numeric image → RGB .visualize()
    minmax = layer.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=region,
        scale=scale,
        maxPixels=1e9
    ).getInfo()

    band = layer.bandNames().getInfo()[0]
    vmin, vmax = minmax[f"{band}_min"], minmax[f"{band}_max"]

    vis_image = layer.visualize(
        min=vmin,
        max=vmax,
        palette=["#000000", "#FFFFFF"]  # placeholder; we'll recolor in PNG step
    )

    # Download as GeoTIFF
    geemap.download_ee_image(
        vis_image,
        region=region,
        filename=tif_path,
        scale=scale,
        crs=crs
    )

    # Convert GeoTIFF to normalized PNG with user-defined colormap
    stretch_and_convert(tif_path, png_path, colormaps.get(name, "viridis"))
    print(f"✅ Saved: {png_path}")


In [ ]:
import cv2
from glob import glob

# Configure video parameters
image_folder = 'exported_maps'
video_name = 'Archaeology_Insights.mp4'
frame_size = (800, 600)
fps = 1  # seconds per frame

image_files = sorted(glob(os.path.join(image_folder, "*.png")))

# Initialize video writer
video_writer = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)

for img_path in image_files:
    img = cv2.imread(img_path)
    resized = cv2.resize(img, frame_size)
    video_writer.write(resized)

video_writer.release()
print(f"🎬 Video created: {video_name}")


In [ ]:
# read known archeological sites
import csv
import numpy as np
with open("/content/Archaeological_Site_Features_2.csv") as infile:
    reader = csv.reader(infile, delimiter=",")
    next(reader, None)
    data3 =np.array(list(reader))

In [ ]:
# Define the type of read parameters for the training of the model
dtype1 = np.dtype([('B12', np.float32),('B11', np.float32),('B8', np.float32),('VV', np.float32),('NDVI', np.float32), ('Elevation', np.float32), ('SM', np.float32), ('ST', np.float32),('ET', np.float32),('Latitude', np.float32),('Longitude', np.float32) , ('Name', np.str_),('Class', np.int32)])
arr2 = np.zeros((len(data3),)).astype(dtype1)

In [ ]:
# assign read data to another array of predefined type dtype1
arr2['B11'] = data3[:, 0]
arr2['B12'] = data3[:, 1]
arr2['B8'] = data3[:, 2]
arr2['ET'] = data3[:, 6]
arr2['NDVI'] = data3[:, 5].astype(np.float32)
arr2['ST'] = data3[:, 8].astype(np.float32)
arr2['VV'] = data3[:, 3].astype(np.float32)
arr2['Elevation'] = data3[:, 4]
arr2['SM'] = data3[:, 7]
arr2['Latitude'] = data3[:, 9]
arr2['Longitude'] = data3[:, 10]
arr2['Name'] = data3[:, 11]
arr2['Class'] = data3[:, 12]

In [ ]:
#create the model
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, BatchNormalization, Input,Reshape
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2
# Define custom loss function (MSE + small L1 penalty)
def custom_loss(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true)) + 0.01 * tf.reduce_sum(tf.abs(y_pred))
def r2_score(y_true, y_pred):
    SS_res = tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - (SS_res / (SS_tot + tf.keras.backend.epsilon()))
# Define feature columns (exclude Name, Latitude, Longitude)
feature_columns = ['B12', 'B11', 'B8', 'VV',  'NDVI', 'Elevation', 'SM', 'ST', 'ET']

X_structured = arr2[feature_columns]  # Extract features
# Extract latitude & longitude properly from the structured array
# Convert latitude & longitude to arcseconds (precision boost)
arr2['Latitude'] = arr2['Latitude'] #* 3600
arr2['Longitude'] = arr2['Longitude']# * 3600
y = np.stack([arr2['Latitude'], arr2['Longitude']], axis=1).astype(np.float32)
X = np.stack([X_structured[col] for col in feature_columns], axis=1).astype(np.float32)
X_lstm1 = X.reshape((X.shape[0], 1, X.shape[1]))
# Normalize features using MinMaxScaler
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)  # Scale features

scaler_y = MinMaxScaler()#feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y)

# Reshape X to fit LSTM input format (samples, timesteps, features)
X_lstm = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # 1 timestep

# Build LSTM model for predicting Latitude & Longitude of possible archeological sites

input_layer = Input(shape=(1, len(feature_columns)))
x = Bidirectional(LSTM(128, return_sequences=True))(input_layer)
x = Dropout(0.3)(x)
x = BatchNormalization()(x)

x = LSTM(64, return_sequences=False)(x)  # Reduce complexity
x = Dropout(0.2)(x)

x = Dense(32, activation='relu')(x)
output_layer = Dense(2, activation='linear')(x)  # Keep it linear for precise regression

model = Model(inputs=input_layer, outputs=output_layer)
# Compile model with MSE loss (regression task)
#model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),loss=custom_loss, metrics= ['mse'])#['mse'])
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss=tf.keras.losses.Huber(delta=0.05),  # More robust loss function
              metrics=['mse'])

#model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=tf.keras.losses.Huber(delta=1.0),metrics= ['mse'])#, metrics=['r2_score'])
# Summary of the model
model.summary()




In [ ]:
# Train the model and plot the graphs of loss function and accuracy
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
history = model.fit(X_lstm1, y, epochs=25, batch_size=16, validation_split=0.25, callbacks=[early_stopping])
# Predict longitude & latitude for new site features
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()

# Plot accuracy using MSE (lower is better)
plt.subplot(1, 2, 2)
plt.plot(history.history['mse'], label='Training mse')
#lt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_mse'], label='Validation mse')
#plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epochs')
#plt.ylabel('Mean Squared Error')
plt.ylabel('mse')
plt.title('Training vs Validation accuracy')
plt.legend()

plt.show()


In [ ]:
#read synthetic data of possible archeological site with specific geographic chracteristics locations are unknown (to find latitude and longitude)
import csv
import numpy as np
with open("/content/Samples_random_collection_2.csv") as infile:
    reader = csv.reader(infile, delimiter=",")
    next(reader, None)
    data4 =np.array(list(reader))

In [ ]:
# another tyep for a new array no location included
dtype3 = np.dtype([('B11', np.float32),('B12', np.float32),('B8', np.float32), ('ET', np.float32),('NDVI', np.float32), ('ST', np.float32),('VV', np.float32),('Elevation', np.float32),('SM', np.float32) ])
arr4 = np.zeros((len(data4),)).astype(dtype3)

In [ ]:
#assign the read attributes from samples file to arr4
arr4['B11'] = data4[:, 0]
arr4['B12'] = data4[:, 1]
arr4['B8'] = data4[:, 2]
arr4['ET'] = data4[:, 3]
arr4['NDVI'] = data4[:, 4].astype(np.float32)
arr4['ST'] = data4[:, 5].astype(np.float32)
arr4['VV'] = data4[:, 6].astype(np.float32)
arr4['Elevation'] = data4[:, 7]
arr4['SM'] = data4[:, 8]

In [ ]:
import pandas as pd
import numpy as np

# Define feature columns
feature_columns = ['B11', 'B12', 'B8','ET',  'NDVI',  'ST','VV', 'Elevation', 'SM']
# Prepare input data
new_data_for_scaling = np.stack([arr4[col] for col in feature_columns], axis=1).astype(np.float32)

# Normalize the new data
#new_scaled = scaler.transform(new_data_for_scaling)

# Reshape for LSTM input format
#new_lstm_ready = new_scaled.reshape((new_scaled.shape[0], 1, new_scaled.shape[1]))
new_lstm_ready = new_data_for_scaling.reshape((new_data_for_scaling.shape[0], 1, new_data_for_scaling.shape[1]))
# Make predictions (now expecting longitude & latitude)
predicted_coords_scaled = model.predict(new_lstm_ready)
#predicted_coords = scaler_y.inverse_transform(predicted_coords_scaled)
# Convert predictions to a DataFrame
results_df = pd.DataFrame(new_data_for_scaling, columns=feature_columns)  # Original input features

# Add predicted longitude & latitude columns
results_df['Predicted_Longitude'] = predicted_coords_scaled[:, 1]  # First column = longitude
results_df['Predicted_Latitude'] = predicted_coords_scaled[:, 0]  # Second column = latitude

# Save results to a CSV file
results_df.to_csv('predicted_coordinates.csv', index=False)

print("Predicted latitude and longitude saved to predicted_coordinates.csv")


In [ ]:
import numpy as np

# True vs predicted coordinates
true_lat = arr2["Latitude"]
true_lon = arr2["Longitude"]
pred_lat = results_df['Predicted_Latitude']#predicted_coords[:, 0]  # Latitude
pred_lon =results_df['Predicted_Longitude']# predicted_coords[:, 1]  # Longitude
print('true latitude',true_lat)
print('true longitude',true_lon)
print('predi latitude',pred_lat)
print('predited longitude',pred_lon)
# Calculate absolute errors for each point
lat_errors = np.abs(true_lat - pred_lat)
lon_errors = np.abs(true_lon- pred_lon)

# Compute mean errors
mean_lat_error = np.mean(lat_errors)
mean_lon_error = np.mean(lon_errors)

print("Individual Latitude Errors:", lat_errors)
print("Mean Latitude Error:", mean_lat_error)
print("Individual Longitude Errors:", lon_errors)
print("Mean Longitude Error:", mean_lon_error)



In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Plot true locations
plt.scatter(arr2["Longitude"], arr2["Latitude"], color='green', label='True Locations', marker='o')

# Plot predicted locations
#plt.scatter(pred_lon[:, 1], pred_lat[:, 0], color='red', label='Predicted Locations', marker='x')
# Plot predicted locations
plt.scatter(pred_lon.values, pred_lat.values, color='red', label='Predicted Locations', marker='x')
# Connect true and predicted points to show displacement
#for i in range(len(true_coords)):
#    plt.plot([true_coords[i, 0], pred_coords[i, 0]], [true_coords[i, 1], pred_coords[i, 1]], color="gray", linestyle="dashed", alpha=0.5)

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("True vs. Predicted Locations of Archaeological Sites")
plt.legend()
plt.grid()
plt.show()

